In [2]:
%pip -q install google-genai
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [3]:
# Instalar Framework ADK de agentes do Google
!pip install -q google-adk

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import requests
import warnings

warnings.filterwarnings("ignore")

In [4]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [12]:
##########################################
# --- Agente 1: Gerador de Ideias --- #
##########################################

def agente_gerador_de_ideias(tipo_do_negocio, descricao_do_negocio, data_de_hoje):
  gerador = Agent(
      name="agente_gerador",
      model="gemini-2.0-flash",
      description="Agente especialista em sugerir ideias",
      tools=[google_search],
      instruction="""
      Você é um especialista em identificar oportunidades de negócios inovadoras e relevantes. Sua missão é gerar um relatório detalhado com ideias para aprimorar o negócio do usuário. Para isso, siga rigorosamente estes passos:
      1. Análise Profunda do Negócio: Utilize a descrição fornecida pelo usuário para compreender a fundo o modelo de negócio atual, seus produtos/serviços, público-alvo e desafios.
      2. Pesquisa Abrangente: Empregue a ferramenta de busca do Google (google_search) para explorar tendências de mercado, novas tecnologias, modelos de negócios disruptivos e cases de sucesso no setor do usuário nos últimos 12 meses.
      3. Geração de Ideias Diversificadas: Apresente uma ampla gama de ideias, desde melhorias incrementais até conceitos completamente novos. Considere diferentes áreas como marketing, vendas, operações, produtos/serviços e experiência do cliente.
      4. Relatório Detalhado: Organize as ideias em um relatório claro e objetivo:
      Seu foco principal é a quantidade e a variedade de ideias promissoras e atuais.
      """
  )


  entrada_do_agente_gerador_de_ideias = f"Tipo do Negócio: {tipo_do_negocio}\nDescrição do Negócio: {descricao_do_negocio}\nData de hoje: {data_de_hoje}"
  relatorio = call_agent(gerador, entrada_do_agente_gerador_de_ideias)
  return relatorio


################################################
# --- Agente 2: Pesquisador --- #
################################################
def agente_pesquisador(localizacao, relatorio):
    pesquisador = Agent(
        name="agente_pesquisador",
        model="gemini-2.0-flash",
        description="Agente que pesquisa informações sobre a cidade do negócio.",
        tools=[google_search],
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um profundo conhecedor da cidade inserida pelo usuário, vivendo e interagindo com sua população há muitos anos.
        Sua tarefa é analisar o relatório de ideias de negócios e filtrar aquelas que possuem maior potencial de sucesso no contexto local. Siga estas etapas:
        1. Imersão na Realidade Local: Utilize a ferramenta de busca do Google (google_search) para investigar a fundo as características únicas da cidade:
        - Cultura e Costumes: Identifique traços culturais marcantes, hábitos da população e tradições locais.
        - Economia: Analise os principais setores econômicos, o poder de compra da população e as tendências econômicas locais.
        - Turismo: Descubra atrações turísticas, eventos sazonais e o perfil dos visitantes.
        - Infraestrutura e Logística: Considere a infraestrutura disponível, desafios logísticos e particularidades geográficas.
        - Eventos e Festivais: Informe-se sobre eventos regulares, festivais culturais e datas importantes para a comunidade.
        2. Avaliação de Viabilidade Local: Analise cada ideia do relatório sob a perspectiva do contexto da cidade. Questione-se sobre:
        - A ideia ressoa com os valores e necessidades da população local?
        - Existe demanda para esse tipo de produto/serviço na cidade?
        - A infraestrutura local suporta a implementação da ideia?
        - Há alguma barreira cultural, econômica ou regulatória para a adoção da ideia?
        3. Filtragem e Justificativa: Mantenha no relatório apenas as ideias que você considera genuinamente viáveis e promissoras para a cidade.
           Para cada ideia mantida, adicione um breve parágrafo justificando sua adequação ao contexto local, mencionando os aspectos culturais, econômicos ou turísticos relevantes.
        Seu foco principal é a relevância e a adaptabilidade das ideias ao cenário específico da cidade.
        """
    )

    entrada_do_agente_pesquisador = f"Localização:{localizacao}\nRelatorio: {relatorio}"
    # Executa o agente
    relatorio_especial = call_agent(pesquisador, entrada_do_agente_pesquisador)
    return relatorio_especial


######################################
# --- Agente 3: Consultor de Negócios --- #
######################################
def agente_consultor_de_negocios(recursos, relatorio_especial):
    consultor = Agent(
        name="agente_consultor_de_negocios",
        model="gemini-2.0-flash",
        description="Consultor de negócios",
        instruction="""
            Você é um consultor de negócios experiente, especializado em identificar oportunidades de crescimento e otimizar o faturamento de empresas de diversos portes e setores.
            Sua responsabilidade é analisar o relatório de ideias filtradas pelo especialista local e selecionar as 3 ideias com maior potencial de implementação,
            considerando os recursos que o usuário provavelmente terá à disposição. Siga este processo:
            1. Análise de Viabilidade Geral: Avalie cada ideia remanescente no relatório sob uma perspectiva de negócios, considerando fatores como:
            - Potencial de geração de receita e lucratividade.
            - Escalabilidade e sustentabilidade a longo prazo.
            - Nível de investimento inicial necessário.
            - Complexidade operacional e de gestão.
            - Riscos e desafios potenciais.
            2. Seleção das 3 Melhores Ideias: Escolha as 3 ideias que você considera mais viáveis e com maior potencial de sucesso para o usuário,
               levando em conta tanto o potencial de mercado quanto a probabilidade de implementação com os recursos disponíveis.
            3. Justificativa da Seleção: Para cada uma das 3 ideias selecionadas, escreva um breve parágrafo explicando os motivos da sua escolha, destacando os pontos fortes da ideia em termos de potencial de faturamento e viabilidade de implementação com recursos prováveis.
            Seu foco principal é a viabilidade prática e o potencial de retorno das ideias, considerando as limitações de recursos.
            OBS: Pode usar todos os recursos em uma ideia apenas, caso você ache necessário.
            """
    )

    entrada_do_agente_consultor_de_negocios = f"Recuros Disponiveis: {recursos}\nRelatorio Especial: {relatorio_especial}"
    # Executa o agente
    rascunho_relatorio_final = call_agent(consultor, entrada_do_agente_consultor_de_negocios)
    return rascunho_relatorio_final


######################################
# --- Agente 4: Consultor Experiente --- #
######################################
def agente_consultor_experiente(rascunho_relatorio_final):
    revisor = Agent(
        name="agente_consultor_experiente",
        model="gemini-2.0-flash",
        description="Agente revisor do conteúdo gerado pelo consultor de negócios",
        instruction="""
            Você é um consultor de negócios com mais de 30 anos de experiência, com um histórico comprovado de sucesso em diversas áreas e empresas de diferentes tamanhos.
            Sua expertise reside na inovação e na implementação eficaz de novas estratégias.
            Sua tarefa é revisar as 3 ideias selecionadas, aprimorá-las com sua visão experiente e fornecer um plano de implementação claro e estruturado. Siga estas etapas:
            1. Revisão Crítica e Aprimoramento: Analise cada uma das 3 ideias selecionadas.
               Identifique possíveis falhas, oportunidades de inovação, sinergias entre as ideias e formas de otimizar cada conceito para maximizar seu impacto. Se necessário, corrija ou refine os detalhes das ideias.
            2. Descrição Detalhada: Para cada uma das 3 ideias, forneça uma descrição final clara, concisa e inspiradora, destacando seu potencial inovador e seus benefícios para o negócio do usuário.
            3. Plano de Implementação Passo a Passo: Elabore um plano de implementação detalhado para cada uma das 3 ideias. O plano deve ser estruturado em etapas sequenciais e fáceis de entender, incluindo:
            - Objetivos Específicos: O que se espera alcançar com a implementação desta etapa.
            - Ações Chave: As tarefas concretas que precisam ser realizadas.
            - Recursos Necessários: Os recursos (humanos, financeiros, materiais, tecnológicos) que serão necessários para cada ação.
            - Prazos Sugeridos: Uma estimativa de tempo para a conclusão de cada etapa.
            - Métricas de Sucesso: Como o progresso e o sucesso da implementação serão medidos.
            Seu foco principal é a inovação, a clareza e a praticidade do plano de implementação, garantindo que o usuário tenha um roteiro claro para colocar as ideias em ação.
            Observação: Não faça tabelas para explicar, explique na forma de texto corrido.
            """
    )

    entrada_do_agente_consultor_experiente = f"Relatório Final: {rascunho_relatorio_final}"
    # Executa o agente
    relatorio_final = call_agent(revisor, entrada_do_agente_consultor_experiente)
    return relatorio_final

In [14]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Assistente de Ideias para Negócios🚀")

# --- Obter o Tópico do Usuário ---
tipo_de_negocio = input("❓ Por favor, digite o Tipo De Negócio sobre o qual você quer gerar novas ideias: ")
descricao_do_negocio = input("❓ Por favor, descreva o seu negócio sobre o qual você quer criar o relatorio de ideias: ")
localizacao = input("❓ Por favor, digite a Localização que está o seu negócio: ")
recursos = input("❓ Por favor, digite os recursos disponiveis que o seu negócio tem: ")

# Inserir lógica do sistema de agentes ################################################
if not tipo_de_negocio:
  print("Você esqueceu de digitar algum tipo de informação, tente novamente!")
else:
  print(f'Maravilha! Vamos então criar o relatório de ideias baseado no seu tipo de negócio: {tipo_de_negocio}')

  relatorio = agente_gerador_de_ideias(tipo_de_negocio, descricao_do_negocio, data_de_hoje)
  print("\n--- Resultado do Agente 1 (Gerador de Ideias) ---\n")
  print(relatorio)
  print("---------------------------------------------------")

  relatorio_especial = agente_pesquisador(localizacao, relatorio)
  print("\n--- Resultado do Agente 2 (Pesquisador) ---\n")
  print(relatorio_especial)
  print("---------------------------------------------------")

  rascunho_relatorio_final = agente_consultor_de_negocios(recursos, relatorio_especial)
  print("\n--- Resultado do Agente 3 (Consultor de Negócios) ---\n")
  print(rascunho_relatorio_final)
  print("---------------------------------------------------")

  relatorio_final = agente_consultor_experiente(rascunho_relatorio_final)
  print("\n--- Resultado do Agente 4 (Consultor de Negócios Experiente) ---\n")
  print(relatorio_final)
  print("---------------------------------------------------")

🚀 Iniciando o Assistente de Ideias para Negócios🚀
❓ Por favor, digite o Tipo De Negócio sobre o qual você quer gerar novas ideias: Oficina mecaninca 
❓ Por favor, descreva o seu negócio sobre o qual você quer criar o relatorio de ideias: especializada em veiculos elétricos
❓ Por favor, digite a Localização que está o seu negócio: pocos de caldas
❓ Por favor, digite os recursos disponiveis que o seu negócio tem: 100 mil reais 
Maravilha! Vamos então criar o relatório de ideias baseado no seu tipo de negócio: Oficina mecaninca 

--- Resultado do Agente 1 (Gerador de Ideias) ---

Ok, entendi. Sou um agente especializado em sugerir ideias para aprimorar negócios. Para te ajudar com a sua oficina mecânica especializada em veículos elétricos, vou gerar um relatório detalhado com diversas ideias inovadoras e relevantes.

Primeiro, farei uma pesquisa abrangente sobre as tendências de mercado, novas tecnologias, modelos de negócios disruptivos e cases de sucesso no setor de veículos elétricos n